<a href="https://colab.research.google.com/github/enesemretas/mcpath-colab/blob/main/notebooks/mcpath_form.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title MCPath (User Interface Only)
# This cell only loads the UI from the GitHub module.
!pip -q install pyyaml ipywidgets requests

import sys, subprocess, pathlib
REPO = "https://github.com/enesemretas/mcpath-colab.git"

# Clone once if not already present
if not pathlib.Path("mcpath-colab").exists():
    subprocess.run(["git", "clone", "--depth", "1", REPO], check=True)

sys.path.append("/content/mcpath-colab")   # so we can import the module
from mcpath.ui import launch

launch(
    defaults_url="https://raw.githubusercontent.com/enesemretas/mcpath-colab/main/config/defaults.yaml",
    show_title="MCPath-style Parameters"
)


In [2]:
from IPython.display import HTML
HTML("""
<style>div.cell.code_cell .input { display: none; }</style>
<button onclick="
  var cells = document.querySelectorAll('div.cell.code_cell .input');
  var anyHidden = Array.from(cells).some(c => c.style.display === 'none');
  for (const c of cells) c.style.display = anyHidden ? '' : 'none';
">Show/Hide code</button>
""")
